# Assignment3
## Autoencoder
### Determinstic autoencoder

In [1]:
# libraries
import torch
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
# local imports
import MNIST_dataloader
import autoencoder_template
import os
import torch.nn.functional as F
# local imports
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
import numpy as np
import matplotlib.cm
import torch.optim as optim

output_notebook()

/Users/lynch_mini/Documents/SML/5LSL0_HW/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BokehJS ...

## Load data MNST

In [2]:
# set torches random seed
torch.random.manual_seed(0)
PATH = os.getcwd()
data_loc = os.path.join(PATH,'data')

# EX1
## AE

In [3]:
# bactch design

batch_size = 64

learning_rate = 3e-4
train_loader, test_loader = MNIST_dataloader.create_dataloaders(data_loc, batch_size)
len(train_loader)

938

## Train model

In [9]:
no_epochs = 5
# device fridenly not at all
if  torch.cuda.is_available():
        device = torch.device('cuda')
else:
    device = torch.device('cpu')
# enumerate(tqdm(train_loader))
# initialize model
AE = autoencoder_template.AE()
AE.to(device) 

optimizer = torch.optim.Adam(AE.parameters(), lr=learning_rate)

train_los = []
for epoch in range(no_epochs):
    print(f"\nTraining Epoch {epoch}:")
    # go over all minibatches
    # len(x_clean) = batch_idx*batch_size
    loss_sum =0 # reset loss
    for batch_idx,(x_clean, x_noisy, label) in enumerate(tqdm(train_loader)):
        # fill in how to train your network using only the clean images
        
        optimizer.zero_grad() # 将模型的参数梯度初始化为0
        # x_clean = x_clean.to(device) # Transfer to GPU
        output,feature = AE(x_clean)
        loss_ = nn.MSELoss()
        loss_(output,x_clean)
        # nn.MSELoss(output,x_clean,label)
        # loss = F.mse_loss(output, x_clean)
        # print(loss)
        loss_(output,x_clean).backward()
        # optimizer
        optimizer.step()
        loss_sum += loss_(output,x_clean).item()
    train_los.append(loss_sum/len(test_loader.dataset.Clean_Images)) # normlaize
    torch.save(AE.state_dict(), os.path.join(data_loc, 'epoch-{}.pt'.format(epoch)))

# # save the trained model
# model.state_dict()

# torch.save(AE.state_dict(), data_loc) # '/Model/state.pt'



Training Epoch 0:


100%|██████████| 938/938 [00:24<00:00, 38.79it/s]



Training Epoch 1:


100%|██████████| 938/938 [00:23<00:00, 39.38it/s]



Training Epoch 2:


100%|██████████| 938/938 [00:25<00:00, 37.11it/s]



Training Epoch 3:


100%|██████████| 938/938 [00:24<00:00, 38.11it/s]



Training Epoch 4:


100%|██████████| 938/938 [00:23<00:00, 39.67it/s]


In [10]:
# plot train loss
p = figure(x_axis_label='Number of epochs',y_axis_label='Train Loss', width=550,height=550,x_range = (0,no_epochs), title='Train loss')
p.line(np.arange(len(train_los)), train_los, legend_label="Train Loss")
show(p,notebook_handle=True)